In [0]:
import cv2 as cv2
import yaml
import numpy as np
import os
import time
import boto3
import tensorflow as tf

In [0]:
COLOR_BLACK = (0, 0, 0)
COLOR_BLUE = (0, 0, 255)
COLOR_GREEN = (0, 255, 0)
COLOR_RED = (255, 0, 0)
COLOR_WHITE = (255, 255, 255)

In [0]:
VIDEO_SOURCE = "/Users/Harshitha/Desktop/Harshitha/Data/version1.mp4"
output="/Users/Harshitha/Desktop/Data/OccupancyDetection_Keras.mp4"
coordinatesFile = "/Users/Harshitha/Desktop/Data/coordinate_keras.yml"
model_path = "/Users/Harshitha/Downloads/Keras_Weights_final.h5"

In [0]:
def show(image):
    plt.figure(figsize=(15, 12))
    plt.imshow(image)
    plt.show()

In [0]:
stream = cv2.VideoCapture(VIDEO_SOURCE)

In [0]:
video_info = {  'fps':    stream.get(cv2.CAP_PROP_FPS),
                'width':  int(stream.get(cv2.CAP_PROP_FRAME_WIDTH)*0.6),
                'height': int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT)*0.6),
                'fourcc': stream.get(cv2.CAP_PROP_FOURCC),
                'num_of_frames': int(stream.get(cv2.CAP_PROP_FRAME_COUNT))}

frame_width = int(stream.get(3))
frame_height = int(stream.get(4))
#print(frame_width,frame_height)

size = (frame_width, frame_height)
#print(size)

outputVideo = cv2.VideoWriter(output,cv2.VideoWriter_fourcc('X','V','I','D'),5,size)

In [0]:
print(video_info)
bounds = []
contours=[]

{'fps': 29.97, 'width': 768, 'height': 432, 'fourcc': 828601953.0, 'num_of_frames': 12738}


In [0]:
with open(coordinatesFile, "r") as data:
    points = yaml.load(data)
    for c in points:
        coordinates=np.array(c["coordinates"])
        rect = cv2.boundingRect(coordinates)
        #print(coordinates)
        new_points = coordinates.copy()
        new_points[:,0] = coordinates[:,0] - rect[0]
        new_points[:,1] = coordinates[:,1] - rect[1]
        contours.append(points)
        bounds.append(rect)

In [0]:
new_model = tf.keras.models.load_model(model_path)
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 34, 34, 128)      

In [0]:
def updateDB(slotnumber,slot_status):
    
    current_time = time.strftime("%m-%d-%Y %H:%M:%S")
    dynamodb = boto3.resource('dynamodb',region_name='us-east-2')
    table = dynamodb.Table('SlotInformation')
    #print(slot_status,current_time)
    table.update_item(
    Key={
        'Slot': slotnumber
    },
    UpdateExpression='SET SlotStatus = :val1',
    ExpressionAttributeValues={
        ':val1': slot_status
    }
    )
    table.update_item(
    Key={
        'Slot': slotnumber
    },
    UpdateExpression='SET SlotTime = :val1',
    ExpressionAttributeValues={
        ':val1': current_time
    }
)

In [0]:
import matplotlib.pyplot as plt
count=0
while stream.isOpened():
    success,image= stream.read()
    if not success:
        print("video done")
        break
    
    if(count%20 == 0):
        
        image_copy = image.copy()
        car_count = 0
        vacant_slots = 0
        class_ids = []
        confidences = []
        boxes = []
        for index, coordinate in enumerate(points):
            carExists = 0
            coordinates=np.array(coordinate["coordinates"])
            slot=np.a0])
            if carExists == 0:
                car_count = car_count +1
                colour = COLOR_RED
                slot_status = 'occupied'
            else:
                vacant_slots = vacant_slots + 1
                colour = COLOR_GREEN
                slot_status = 'Empty'
            slotnumber = coordinate["rray(coordinate["id"])
            rect = bounds[index]
            roi_gray = image[rect[1]:(rect[1]+rect[3]), rect[0]:(rect[0]+rect[2])]
            (H, W) = roi_gray.shape[:2]
            roi_gray = tf.image.resize(roi_gray, [150, 150])
            img = (np.expand_dims(roi_gray,0))
            predictions = new_model.predict(img)
            carExists = np.argmax(predictions[id"]
            updateDB(slotnumber, slot_status)
            cv2.drawContours(image_copy, [coordinates], contourIdx=-1,color=colour, thickness=2, lineType=cv2.LINE_8)
            moments = cv2.moments(coordinates)
            center = (int(moments["m10"] / moments["m00"]) - 3,
                         int(moments["m01"] / moments["m00"]) + 3)
            label = str(coordinate["id"])
            cv2.putText(image_copy,label,center,cv2.FONT_HERSHEY_SIMPLEX,1,COLOR_WHITE,2,cv2.LINE_AA)


        vacant_label = str("Vacant:")+ str(vacant_slots) 
        car_label = str("Total Cars:") + str(car_count)
        cv2.putText(image_copy,vacant_label,(30,50),cv2.FONT_HERSHEY_SIMPLEX,1,COLOR_RED,2,cv2.LINE_AA) 
        cv2.putText(image_copy,car_label,(30,90),cv2.FONT_HERSHEY_SIMPLEX,1,COLOR_RED,2,cv2.LINE_AA)


        #show(image_copy)
        print(count)
        outputVideo.write(image_copy)

    #if count==10:
        #break
    count = count + 1 
    
stream.release()
outputVideo.release()
cv2.destroyAllWindows()



0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640
2670
2700
2730
2760
2790
2820
2850
2880
2910
2940
2970
3000
3030
3060
3090
3120
3150
3180
3210
3240
3270
3300
3330
3360
3390
3420
3450
3480
3510
3540
3570
3600
3630
3660
3690
3720
3750
3780
3810
3840
3870
3900
3930
3960
3990
4020
4050
4080
4110
4140
4170
4200
4230
4260
4290
4320
4350
4380
4410
4440
4470
4500
4530
4560
4590
4620
4650
4680
4710
4740
4770
4800
4830
4860
4890
4920
4950
4980
5010
5040
5070
5100
5130
5160
5190
5220
5250
5280
5310
5340
5370
5400
5430
5460
5490
5520
5550
5580
5610
5640
5670
5700
5730
5760
5790
5820
5850
5880
5910
5940
5970
6000
6030
6060
6090
6120
6150
6180
6210